## RAW DATA TO MONGODB STAGING

In [ ]:
import json
import pprint
import numpy as np
import pandas as pd
from datetime import datetime

import pymongo
from pymongo import MongoClient

# import findspark
# findspark.init('C:/BigDataLocalSetup/spark')
import pyspark
from pyspark import SparkContext
from pyspark.sql.functions import *
from pyspark.sql import SparkSession

spark = SparkSession.\
        builder.\
        appName("pyspark-notebook2").\
        config("spark.executor.memory", "1g").\
        config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.0").\
        getOrCreate()

In [ ]:
client = MongoClient("mongodb://127.0.0.1:27017")
db = client.MED_EX_CHAIN

gpo = db.GPO
product = db.PRODUCT
hospital = db.HOSPITAL
user_roles = db.USER_ROLES
distributor = db.DISTRIBUTOR
manufacturer = db.MANUFACTURER
correlation = db.CORRELATION_DATA
product_similarity = db.PRODUCT_SIMILARITY

In [ ]:
raw_data_path = "../Data/SampleData/Schema - Master Data/"

### Distributor Raw Staging

In [ ]:
distributor.insert_one({'DISTRIBUTOR_ID': 0, 'VERSION': 0})

In [ ]:
file_name = 'Distributor_update.csv'
# file_name = input('Enter File Name: ')
distributor_csv = spark.read.csv(raw_data_path + file_name, header = True).dropna(how = "all")
# distributor_csv.show()

In [ ]:
print(distributor_csv.toJSON().collect()[0])

In [ ]:
distributor_data = spark.read.format("mongo").option("uri", 
                                "mongodb://127.0.0.1:27017/MED_EX_CHAIN.DISTRIBUTOR").load()
distributor_data.createOrReplaceTempView('distributor_view')

In [ ]:
for i in distributor_csv.toJSON().collect():
    new_record = json.loads(i)
    distributor_id = new_record['DISTRIBUTOR_ID']
    
    query = "select * from distributor_view \
             where DISTRIBUTOR_ID = '" + distributor_id + "' and\
                   VERSION = (select max(VERSION) from distributor_view \
             where DISTRIBUTOR_ID = '" + distributor_id + "')"

    available_data = spark.sql(query)
    print(distributor_id, available_data.count())
    
    if available_data.count() == 0: # no data available, insert with version 1.0
        insert_data = dict(new_record)
        insert_data['VERSION'] = 1.0
        insert_data['START_DATE'] = str(datetime.now())
        insert_data['END_DATE'] = str('9999-12-31')
        insert_data['LAST_UPDATE_DT'] = str(datetime.now())
        distributor.insert_one(insert_data)
        print('insert data with 1 version.')
        
    elif (new_record['DISTRIBUTOR_NAME'] != available_data.rdd.collect()[0]['DISTRIBUTOR_NAME'])|\
         (new_record['DISTRIBUTOR_ADDRESS'] != available_data.rdd.collect()[0]['DISTRIBUTOR_ADDRESS'])|\
         (new_record['DISTRIBUTOR_SHORT_NAME'] != available_data.rdd.collect()[0]['DISTRIBUTOR_SHORT_NAME']): 
         # add conditions for all the columns which are eligible for the SCD.
         
         distributor.update_many({'DISTRIBUTOR_ID': distributor_id, 
                                  'VERSION': int(available_data.collect()[0].asDict()['VERSION'])},
                                 {"$set": {'LAST_UPDATE_DT': str(datetime.now()),
                                           'END_DATE': str(datetime.now())}})
        

         insert_data = dict(new_record)
         insert_data['START_DATE'] = str(datetime.now())
         insert_data['END_DATE'] = str('9999-12-31')
         insert_data['LAST_UPDATE_DT'] = str(datetime.now())
         insert_data['VERSION'] = int(available_data.collect()[0].asDict()['VERSION']) + 1
         distributor.insert_one(insert_data)
         print('insert data with updated + 1 version.')
                
    # insert data with new version.             
    else:
        update_data = dict(new_record)
        update_data['LAST_UPDATE_DT'] = str(datetime.now())
        distributor.update_one({'DISTRIBUTOR_ID': distributor_id, 
                                'VERSION': int(available_data.collect()[0].asDict()['VERSION'])},
                            {"$set": update_data})
        print('update record with same latest version')  # update existing data with same version.


In [ ]:
distributor_data = spark.read.format("mongo").option("uri",
                                "mongodb://127.0.0.1:27017/MED_EX_CHAIN.DISTRIBUTOR").load()
# distributor_data.show()                            

### GPO Raw staging

In [ ]:
gpo.insert_one({'GPO_ID':0, 'VERSION': 0})

In [ ]:
file_name = 'GPO_update.csv'
# file_name = input('Enter File Name: ')
gpo_csv = spark.read.csv(raw_data_path + file_name, header = True).dropna(how = "all")
# gpo_csv.show()

In [ ]:
gpo_data = spark.read.format("mongo").option("uri",
                                "mongodb://127.0.0.1:27017/MED_EX_CHAIN.GPO").load()
gpo_data.createOrReplaceTempView('gpo_view')

In [ ]:
for i in gpo_csv.toJSON().collect():
    new_record = json.loads(i)
    gpo_id = new_record['GPO_ID']

    query = "select * from gpo_view \
             where GPO_ID = '" + gpo_id + "' and\
                   VERSION = (select max(VERSION) from gpo_view \
             where GPO_ID = '" + gpo_id + "')"
    
    available_data = spark.sql(query)
    print(gpo_id, available_data.count())
    
    if available_data.count() == 0: # no data available, insert with version 1.0
        insert_data = dict(new_record)
        insert_data['VERSION'] = 1.0
        insert_data['START_DATE'] = str(datetime.now())
        insert_data['END_DATE'] = str('9999-12-31')
        insert_data['LAST_UPDATE_DT'] = str(datetime.now())
        gpo.insert_one(insert_data)
        print('insert data with 1 version.')
        
    elif (new_record['GPO_NAME'] != available_data.rdd.collect()[0]['GPO_NAME'])|\
         (new_record['GPO_ADDRESS'] != available_data.rdd.collect()[0]['GPO_ADDRESS'])|\
         (new_record['GPO_SHORT_NAME'] != available_data.rdd.collect()[0]['GPO_SHORT_NAME']): 
         # add conditions for all the columns which are eligible for the SCD.

         gpo.update_many({'GPO_ID': gpo_id, 
                          'VERSION': int(available_data.collect()[0].asDict()['VERSION'])},
                         {"$set": {'LAST_UPDATE_DT': str(datetime.now()),
                                   'END_DATE': str(datetime.now())}})

         insert_data = dict(new_record)
         insert_data['START_DATE'] = str(datetime.now())
         insert_data['END_DATE'] = str('9999-12-31')
         insert_data['LAST_UPDATE_DT'] = str(datetime.now())
         insert_data['VERSION'] = int(available_data.collect()[0].asDict()['VERSION']) + 1
         gpo.insert_one(insert_data)
         print('insert data with updated + 1 version.')
                
    # insert data with new version.                 
    else:
        update_data = dict(new_record)
        update_data['LAST_UPDATE_DT'] = str(datetime.now())
        gpo.update_one({'GPO_ID': gpo_id, 
                             'VERSION': int(available_data.collect()[0].asDict()['VERSION'])},
                             {"$set": update_data})
        print('update record with same latest version')  # update existing data with same version.


In [ ]:
gpo_data = spark.read.format("mongo").option("uri",
                                "mongodb://127.0.0.1:27017/MED_EX_CHAIN.GPO").load()
gpo_data.show()                            

### Hospital Raw Staging

In [ ]:
hospital.insert_one({'HOSPITAL_ID':0, 'VERSION': 0})

In [ ]:
file_name = 'Hospital.csv'
# file_name = input('Enter File Name: ')
hospital_csv = spark.read.csv(raw_data_path + file_name, header = True).dropna(how = "all")
# hospital_csv.show()

In [ ]:
hospital_data = spark.read.format("mongo").option("uri",
                                "mongodb://127.0.0.1:27017/MED_EX_CHAIN.HOSPITAL").load()
hospital_data.createOrReplaceTempView('hospital_view')

In [ ]:
for i in hospital_csv.toJSON().collect():
    new_record = json.loads(i)
    hospital_id = new_record['HOSPITAL_ID']

    query = "select * from hospital_view \
             where HOSPITAL_ID = '" + hospital_id + "' and\
                   VERSION = (select max(VERSION) from hospital_view \
             where HOSPITAL_ID = '" + hospital_id + "')"
    
    available_data = spark.sql(query)
    print(hospital_id, available_data.count())
    
    if available_data.count() == 0: # no data available, insert with version 1.0
        insert_data = dict(new_record)
        insert_data['VERSION'] = 1.0
        insert_data['START_DATE'] = str(datetime.now())
        insert_data['END_DATE'] = str('9999-12-31')
        insert_data['LAST_UPDATE_DT'] = str(datetime.now())
        hospital.insert_one(insert_data)
        print('insert data with 1 version.')
        
    elif (new_record['HOSPITAL_NAME'] != available_data.rdd.collect()[0]['HOSPITAL_NAME'])|\
         (new_record['HOSPITAL_ADDRESS'] != available_data.rdd.collect()[0]['HOSPITAL_ADDRESS'])|\
         (new_record['HOSPITAL_SHORT_NAME'] != available_data.rdd.collect()[0]['HOSPITAL_SHORT_NAME']): 
         # add conditions for all the columns which are eligible for the SCD.
         
         hospital.update_many({'HOSPITAL_ID': hospital_id, 
                               'VERSION': int(available_data.collect()[0].asDict()['VERSION'])},
                              {"$set": {'LAST_UPDATE_DT': str(datetime.now()),
                                        'END_DATE': str(datetime.now())}})

         insert_data = dict(new_record)
         insert_data['START_DATE'] = str(datetime.now())
         insert_data['END_DATE'] = str('9999-12-31')
         insert_data['LAST_UPDATE_DT'] = str(datetime.now())
         insert_data['VERSION'] = int(available_data.collect()[0].asDict()['VERSION']) + 1
         hospital.insert_one(insert_data)
         print('insert data with updated + 1 version.')
                
    # insert data with new version.                
    else:
        update_data = dict(new_record)
        update_data['LAST_UPDATE_DT'] = str(datetime.now())
        hospital.update_one({'HOSPITAL_ID': hospital_id, 
                                'VERSION': int(available_data.collect()[0].asDict()['VERSION'])},
                                 {"$set": update_data})
        print('update record with same latest version')  # update existing data with same version.


In [ ]:
hospital_data = spark.read.format("mongo").option("uri",
                                "mongodb://127.0.0.1:27017/MED_EX_CHAIN.HOSPITAL").load()
hospital_data.show()                             


### Manufacturer Raw Staging

In [ ]:
manufacturer.insert_one({'MANUFACTURER_ID':0, 'VERSION': 0})

In [ ]:
file_name = 'Manufacturer.csv'
# file_name = input('Enter File Name: ')
manufacturer_csv = spark.read.csv(raw_data_path + file_name, header = True).dropna(how = "all")
# manufacturer_csv.show()

In [ ]:
manufacturer_data = spark.read.format("mongo").option("uri",
                                "mongodb://127.0.0.1:27017/MED_EX_CHAIN.MANUFACTURER").load()
manufacturer_data.createOrReplaceTempView('manufacturer_view')

In [ ]:
for i in manufacturer_csv.toJSON().collect():
    new_record = json.loads(i)
    manufacturer_id = new_record['MANUFACTURER_ID']

    query = "select * from manufacturer_view \
             where MANUFACTURER_ID = '" + manufacturer_id + "' and \
                   VERSION = (select max(VERSION) from manufacturer_view \
             where MANUFACTURER_ID = '" + manufacturer_id + "')"
    
    available_data = spark.sql(query)
    print(manufacturer_id, available_data.count())
    
    if available_data.count() == 0: # no data available, insert with version 1.0
        insert_data = dict(new_record)
        insert_data['VERSION'] = 1.0
        insert_data['START_DATE'] = str(datetime.now())
        insert_data['END_DATE'] = str('9999-12-31')
        insert_data['LAST_UPDATE_DT'] = str(datetime.now())
        manufacturer.insert_one(insert_data)
        print('insert data with 1 version.')
        
    elif (new_record['MANUFACTURER_NAME'] != available_data.rdd.collect()[0]['MANUFACTURER_NAME'])|\
         (new_record['MANUFACTURER_ADDRESS'] != available_data.rdd.collect()[0]['MANUFACTURER_ADDRESS'])|\
         (new_record['MANUFACTURER_SHORT_NAME'] != available_data.rdd.collect()[0]['MANUFACTURER_SHORT_NAME']): 
         # add conditions for all the columns which are eligible for the SCD.
         
         manufacturer.update_many({'MANUFACTURER_ID': manufacturer_id , 
                                   'VERSION': int(available_data.collect()[0].asDict()['VERSION'])},
                                  {"$set": {'LAST_UPDATE_DT': str(datetime.now()),
                                            'END_DATE': str(datetime.now())}})
        
         insert_data = dict(new_record)
         insert_data['START_DATE'] = str(datetime.now())
         insert_data['END_DATE'] = str('9999-12-31')
         insert_data['LAST_UPDATE_DT'] = str(datetime.now())
         insert_data['VERSION'] = int(available_data.collect()[0].asDict()['VERSION']) + 1
         manufacturer.insert_one(insert_data)
         print('insert data with updated + 1 version.')
                
    # insert data with new version.                 
    else:
        update_data = dict(new_record)
        update_data['LAST_UPDATE_DT'] = str(datetime.now())
        manufacturer.update_one({'MANUFACTURER_ID': manufacturer_id, 
                                 'VERSION': int(available_data.collect()[0].asDict()['VERSION'])},
                                {"$set": update_data})
        print('update record with same latest version')  # update existing data with same version.

In [ ]:
manufacturer_data = spark.read.format("mongo").option("uri",
                                "mongodb://127.0.0.1:27017/MED_EX_CHAIN.MANUFACTURER").load()
manufacturer_data.show()                                

### User-Roles Raw Staging

In [ ]:
user_roles.insert_one({'USER_ID': 0, 'VERSION': 0})

In [ ]:
file_name = 'User.csv'
# file_name = input('Enter File Name: ')
user_csv = spark.read.csv(raw_data_path + file_name, header = True).dropna(how = "all")
# user_csv.show()

In [ ]:
user_data = spark.read.format("mongo").option("uri",
                            "mongodb://127.0.0.1:27017/MED_EX_CHAIN.USER_ROLES").load()
user_data.createOrReplaceTempView('user_view')

In [ ]:
for i in user_csv.toJSON().collect():
    new_record = json.loads(i)
    user_id = new_record['USER_ID']

    query = "select * from user_view \
             where USER_ID = '" + user_id + "' and\
                   VERSION = (select max(VERSION) from user_view \
             where USER_ID = '" + user_id + "')"
    
    available_data = spark.sql(query)
    print(user_id, available_data.count())
    
    if available_data.count() == 0: # no data available, insert with version 1.0
        insert_data = dict(new_record)
        insert_data['VERSION'] = 1.0
        insert_data['START_DATE'] = str(datetime.now())
        insert_data['END_DATE'] = str('9999-12-31')
        insert_data['LAST_UPDATE_DT'] = str(datetime.now())
        user_roles.insert_one(insert_data)
        print('insert data with 1 version.')
        
    elif (new_record['FIRST_NAME'] != available_data.rdd.collect()[0]['FIRST_NAME'])|\
         (new_record['USER_ADDRESS'] != available_data.rdd.collect()[0]['USER_ADDRESS'])|\
         (new_record['PHONE_NUMBER'] != available_data.rdd.collect()[0]['PHONE_NUMBER'])|\
	     (new_record['EMAIL_ID'] != available_data.rdd.collect()[0]['EMAIL_ID']): 
         # add conditions for all the columns which are eligible for the SCD.

         user_roles.update_many({'USER_ID': user_id, 
                                 'VERSION': int(available_data.collect()[0].asDict()['VERSION'])},
                                  {"$set": {'LAST_UPDATE_DT': str(datetime.now()),
                                            'END_DATE': str(datetime.now())}})
         insert_data = dict(new_record)
         insert_data['START_DATE'] = str(datetime.now())
         insert_data['END_DATE'] = str('9999-12-31')
         insert_data['LAST_UPDATE_DT'] = str(datetime.now())
         insert_data['VERSION'] = int(available_data.collect()[0].asDict()['VERSION']) + 1
         user_roles.insert_one(insert_data)
         print('insert data with updated + 1 version.')
                
    # insert data with new version.                 
    else:
        update_data = dict(new_record)
        update_data['LAST_UPDATE_DT'] = str(datetime.now())
        user_roles.update_one({'USER_ID': user_id, 
                                'VERSION': int(available_data.collect()[0].asDict()['VERSION'])},
                                 {"$set": update_data})
        print('update record with same latest version')  # update existing data with same version.


In [ ]:
user_data = spark.read.format("mongo").option("uri",
                            "mongodb://127.0.0.1:27017/MED_EX_CHAIN.USER_ROLES").load()
# user_data.show()                            

## CATALOG DATA TO MONGODB STAGING

In [ ]:
product_catalog_data_path = "../Data/SampleData/Product Data"

### Distributor Product Catalog

In [ ]:
distributor_file = 'Distributor-CardinalHealth_Product_Catalog.xlsx'
# distributor_file = input('Enter distributor product catalog file name: ')
distributor_catalog = pd.read_excel(product_catalog_data_path + "/Distributor/" + distributor_file).dropna(how = 'all').astype(str)

# ERRORRR

###  we have defined same varibale dataframe "distributer_catalog" above and below with different files: so dont run below code

In [ ]:
distributor_catalog = pd.read_excel(product_catalog_data_path + "/Distributor/Mckesson_Product_Catalog.xlsx", 
                                    sheet_name = 'Charting Paper Detail').dropna(how = 'all').astype(str)

In [ ]:
distr_product_catalog = distributor_catalog[['Account Name', 'Supplier Name', 'Supplier SKU Number', 
                                             'Distributor SKU', 'SKU Description', 'Hospital Item Number','Distributor Name', 'Contract Price', 'Contract Number', 'Unit Price']]

distr_product_catalog = distr_product_catalog.rename(columns = {'Account Name': 'HOSPITAL_NAME',
                                                                'Supplier Name': 'MANUFACTURER_NAME',
                                                                'Supplier SKU Number': 'PRODUCT_ID',
                                                                'Distributor SKU': 'DISTRIBUTOR_SKU',
                                                                'SKU Description': 'DESCRIPTION',
                                                                'Hospital Item Number': 'HOSPITAL_SKU',
                                                                'Distributor Name': 'DISTRIBUTOR_NAME',
                                                                'Contract Price':'CONTRACT_PRICE',
                                                                'Contract Number':'CONTRACT_NUMBER',
                                                                'Unit Price':'BASE_PRICE'})

correlation_data = distr_product_catalog[['PRODUCT_ID', 'DISTRIBUTOR_NAME', 'DISTRIBUTOR_SKU', 'HOSPITAL_NAME','HOSPITAL_SKU', 'MANUFACTURER_NAME', 'DESCRIPTION', 'CONTRACT_PRICE',    'CONTRACT_NUMBER','BASE_PRICE']]


correlation.insert_many(json.loads(correlation_data.to_json(orient = 'records')))

### Manufacturer Product Catalog

In [ ]:
product.insert_one({'PRODUCT_ID': 0, 'VERSION': 0})

In [ ]:
# file_name = 'Medical Action Industries_product_catalog_1.csv'
# file_name = 'MALLINCKRODT_product_catalog.csv'
file_name = 'Manufacturer-ProductSimilarity_Catalogue.csv'

# file_name = input('Enter Manufacturer product catalog File Name: ')
manufacturer_catalog = spark.read.csv(product_catalog_data_path + "/Manufacturer/" + file_name, 
                                        header = True).dropna(how = 'all')
# manufacturer_catalog.show()

In [ ]:
manufacturer_catalog = manufacturer_catalog.select(col('Item number'),
                                                   col('Manufacturer_Catalog_Number'),
                                                   col('Description'),
                                                   col('Size'),
                                                   col('Gauge'),
                                                   col('Color'),
                                                   col('BUOM'),
                                                   col('UM'),
                                                   col('Base Price'),
                                                   col(' PRODUCT_TYPE_ID '),
                                                   col(' NAME '),
                                                   col(' SHORT_DESCRIPTION '),
                                                   col(' PRODUCT_STATUS '),
                                                   col(' IS_PACK '),
                                                   col(' UNITS_IN_PACK '),
                                                   col(' PRODUCT_CATEGORY '),
                                                   col(' MFG_DATE '),
                                                   col(' EXPIRY_DATE '),
                                                   col(' PRODUCT_IMAGE '),
                                                   col('Manufacturer'))

In [ ]:
manufacturer_catalog = manufacturer_catalog.withColumnRenamed('Item number', 'PRODUCT_ID').\
                                            withColumnRenamed('Description', 'DESCRIPTION').\
                                            withColumnRenamed('Manufacturer', 'MANUFACTURER').\
                                            withColumnRenamed('Size', 'SIZE').\
                                            withColumnRenamed('Manufacturer_Catalog_Number', 'CATALOG_NUMBER').\
                                            withColumnRenamed('Gauge', 'VOLUME').\
                                            withColumnRenamed('Color', 'COLOR').\
                                            withColumnRenamed('BUOM', 'WEIGHT_UOM').\
                                            withColumnRenamed('UM', 'UOM').\
                                            withColumnRenamed('Base Price', 'BASE_PRICE').\
                                            withColumnRenamed(' PRODUCT_TYPE_ID ', 'PRODUCT_TYPE_ID').\
                                            withColumnRenamed(' NAME ', 'NAME').\
                                            withColumnRenamed(' SHORT_DESCRIPTION ', 'SHORT_DESCRIPTION').\
                                            withColumnRenamed(' PRODUCT_STATUS ', 'PRODUCT_STATUS').\
                                            withColumnRenamed(' IS_PACK ', 'IS_PACK').\
                                            withColumnRenamed(' UNITS_IN_PACK ', 'UNITS_IN_PACK').\
                                            withColumnRenamed(' PRODUCT_CATEGORY ', 'PRODUCT_CATEGORY').\
                                            withColumnRenamed(' MFG_DATE ', 'MFG_DATE').\
                                            withColumnRenamed(' EXPIRY_DATE ', 'EXPIRY_DATE').\
                                            withColumnRenamed(' PRODUCT_IMAGE ', 'PRODUCT_IMAGE')

In [ ]:
product_data = spark.read.format("mongo").option("uri",
                                "mongodb://127.0.0.1:27017/MED_EX_CHAIN.PRODUCT").load()
product_data.createOrReplaceTempView('product_view')

In [ ]:
for i in manufacturer_catalog.toJSON().collect():
    new_record = json.loads(i)
    product_id = new_record['PRODUCT_ID']

    query = "select * from product_view \
             where PRODUCT_ID = '" + product_id + "' and \
                   VERSION = (select max(VERSION) from product_view \
             where PRODUCT_ID = '" + product_id + "')"
    
    available_data = spark.sql(query)
    print(product_id, available_data.count())
    
    # no data available, insert with version 1.0
    if available_data.count() == 0: 
        insert_data = dict(new_record)
        insert_data['VERSION'] = 1.0
        insert_data['START_DATE'] = str(datetime.now())
        insert_data['END_DATE'] = str('9999-12-31')
        insert_data['LAST_UPDATE_DT'] = str(datetime.now())
        product.insert_one(insert_data)
        print('insert data with 1 version.')
    
    # insert data with new version.    
    elif (new_record['NAME'] != available_data.rdd.collect()[0]['NAME'])|\
         (new_record['DESCRIPTION'] != available_data.rdd.collect()[0]['DESCRIPTION'])|\
         (new_record['BASE_PRICE'] != available_data.rdd.collect()[0]['BASE_PRICE'])|\
         (new_record['CATALOG_NUMBER'] != available_data.rdd.collect()[0]['CATALOG_NUMBER'])|\
         (new_record['UNITS_IN_PACK'] != available_data.rdd.collect()[0]['UNITS_IN_PACK'])|\
         (new_record['SHORT_DESCRIPTION'] != available_data.rdd.collect()[0]['SHORT_DESCRIPTION']): 
         # add conditions for all the columns which are eligible for the SCD.

         product.update_many({'PRODUCT_ID': product_id,
                              'VERSION': int(available_data.collect()[0].asDict()['VERSION'])},
                             {"$set": {'LAST_UPDATE_DT': str(datetime.now()),
                                       'END_DATE':str(datetime.now())}})

         insert_data = dict(new_record)
         insert_data['START_DATE'] = str(datetime.now())
         insert_data['END_DATE'] = str('9999-12-31')
         insert_data['LAST_UPDATE_DT'] = str(datetime.now())
         insert_data['VERSION'] = int(available_data.collect()[0].asDict()['VERSION']) + 1
         product.insert_one(insert_data)
         print('insert data with updated + 1 version.')
                
                
    else:
        update_data = dict(new_record)
        update_data['LAST_UPDATE_DT'] = str(datetime.now())
        product.update_one({'PRODUCT_ID': product_id, 
                            'VERSION': int(available_data.collect()[0].asDict()['VERSION'])},
                           {"$set": update_data})
        print('update record with same latest version')  # update existing data with same version.


In [ ]:
product_data = spark.read.format("mongo").option("uri",
                                "mongodb://127.0.0.1:27017/MED_EX_CHAIN.PRODUCT").load()
product_data.show()                                

## MONGODB STAGING TO NEO4J


CONNECTING TO NEO4J DB

In [ ]:
from py2neo import *

graph = Graph("bolt://localhost:7687", auth = ("neo4j", "shubham"))

READ DATA FROM MONGODB

In [ ]:
df_gpo = spark.read.format("mongo").option("uri",
                                "mongodb://127.0.0.1:27017/MED_EX_CHAIN.GPO").load()
df_product = spark.read.format("mongo").option("uri",
                                "mongodb://127.0.0.1:27017/MED_EX_CHAIN.PRODUCT").load()
df_hospital = spark.read.format("mongo").option("uri",
                                "mongodb://127.0.0.1:27017/MED_EX_CHAIN.HOSPITAL").load()
df_user_roles = spark.read.format("mongo").option("uri",
                                "mongodb://127.0.0.1:27017/MED_EX_CHAIN.USER_ROLES").load()    
df_distributor = spark.read.format("mongo").option("uri",
                                "mongodb://127.0.0.1:27017/MED_EX_CHAIN.DISTRIBUTOR").load()
df_manufacturer = spark.read.format("mongo").option("uri",
                                "mongodb://127.0.0.1:27017/MED_EX_CHAIN.MANUFACTURER").load()
df_correlation_data = spark.read.format("mongo").option("uri",
                                "mongodb://127.0.0.1:27017/MED_EX_CHAIN.CORRELATION_DATA").load()
df_product_similarity = spark.read.format("mongo").option("uri",
                                "mongodb://127.0.0.1:27017/MED_EX_CHAIN.PRODUCT_SIMILARITY").load() 

MAKE VIEWS 

In [ ]:
df_gpo.createOrReplaceTempView('gpo_view')
df_user_roles.createOrReplaceTempView('user_view')
df_product.createOrReplaceTempView('product_view')
df_hospital.createOrReplaceTempView('hospital_view')
df_distributor.createOrReplaceTempView('distributor_view')
df_manufacturer.createOrReplaceTempView('manufacturer_view')


GET DATA FROM VIEW

In [ ]:
df_gpo = spark.sql('select * from gpo_view where (GPO_ID, VERSION) in \
                   (select GPO_ID, max(VERSION) from gpo_view group by GPO_ID)')

df_user_roles = spark.sql('select * from user_view where (USER_ID, VERSION) in \
                          (select USER_ID, max(VERSION) from user_view group by USER_ID)')

df_product = spark.sql('select * from product_view where (PRODUCT_ID, VERSION) in \
                     (select PRODUCT_ID, max(VERSION) from product_view group by PRODUCT_ID)')

df_hospital = spark.sql('select * from hospital_view where (HOSPITAL_ID, VERSION) in \
                        (select HOSPITAL_ID, max(VERSION) from hospital_view group by HOSPITAL_ID)')

df_distributor = spark.sql('select * from distributor_view where (DISTRIBUTOR_ID, VERSION) in \
                           (select DISTRIBUTOR_ID, max(VERSION) from distributor_view group by DISTRIBUTOR_ID)')

df_manufacturer = spark.sql('select * from manufacturer_view where (MANUFACTURER_ID, VERSION) in \
                            (select MANUFACTURER_ID, max(VERSION) from manufacturer_view group by MANUFACTURER_ID)')

Create Product Node

In [ ]:
print(df_product.rdd.collect()[1])

In [ ]:
for record in df_product.rdd.collect():
    graph.run("merge (p: Product{product_id: $product_id})\
                            on create set \
                                    p.product_category = $product_category,\
                                    p.base_price = $base_price,\
                                    p.name = $name,\
                                    p.color = $color,\
                                    p.manufacturer = $manufacturer,\
                                    p.description = $description,\
                                    p.mfg_date = $mfg_date,\
                                    p.expiry_date = $expiry_date,\
                                    p.is_pack = $is_pack,\
                                    p.product_status = $product_status,\
                                    p.product_type_id = $product_type_id,\
                                    p.size = $size,\
                                    p.short_description = $short_description,\
                                    p.units_in_pack = $units_in_pack,\
                                    p.uom = $uom,\
                                    p.volume = $volume,\
                                    p.weight_uom = $weight_uom,\
                                    p.product_image = $product_image,\
                                    p.start_date = $start_date,\
                                    p.end_date = $end_date,\
                                    p.last_update_date = $last_update_date,\
                                    p.version = $version\
                            on match set\
                                    p.product_category = $product_category,\
                                    p.base_price = $base_price,\
                                    p.name = $name,\
                                    p.color = $color,\
                                    p.manufacturer = $manufacturer,\
                                    p.description = $description,\
                                    p.mfg_date = $mfg_date,\
                                    p.expiry_date = $expiry_date,\
                                    p.is_pack = $is_pack,\
                                    p.product_status = $product_status,\
                                    p.product_type_id = $product_type_id,\
                                    p.size = $size,\
                                    p.short_description = $short_description,\
                                    p.units_in_pack = $units_in_pack,\
                                    p.uom = $uom,\
                                    p.volume = $volume,\
                                    p.weight_uom = $weight_uom,\
                                    p.product_image = $product_image,\
                                    p.start_date = $start_date,\
                                    p.end_date = $end_date,\
                                    p.last_update_date = $last_update_date,\
                                    p.version = $version",
             product_id = record['PRODUCT_ID'],
             product_category = record['PRODUCT_CATEGORY'],
             base_price = record['BASE_PRICE'],
             name = record['NAME'],
             color = record['COLOR'],
             manufacturer = record['MANUFACTURER'],
             description = record['DESCRIPTION'],
             mfg_date = record['MFG_DATE'],
             expiry_date = record['EXPIRY_DATE'],
             is_pack = record['IS_PACK'],
             product_status = record['PRODUCT_STATUS'],
             product_type_id = record['PRODUCT_TYPE_ID'],
             size = record['SIZE'],
             short_description = record['SHORT_DESCRIPTION'],
             units_in_pack = record['UNITS_IN_PACK'],
             uom = record['UOM'], 
             volume = record['VOLUME'],
             weight_uom = record['WEIGHT_UOM'],
             product_image = record['PRODUCT_IMAGE'],
             start_date = record['START_DATE'],
             end_date = record['END_DATE'],
             last_update_date = record['LAST_UPDATE_DT'],
             version = record['VERSION'])

Create Manufacturer Node

In [ ]:
for record in df_manufacturer.rdd.collect():
    graph.run("merge (m: Manufacturer{manufacturer_id: $manufacturer_id})\
                                on create set\
                                      m.country_of_origin = $country_of_origin,\
                                      m.manufacturer_name = $manufacturer_name,\
                                      m.manufacturer_address = $manufacturer_address,\
                                      m.manufacturer_org_type = $manufacturer_org_type,\
                                      m.manufacturer_short_name = $manufacturer_short_name,\
                                      m.source_identifier = $source_identifier,\
                                      m.start_date = $start_date,\
                                      m.end_date = $end_date,\
                                      m.last_update_date = $last_update_date,\
                                      m.last_update_user = $last_update_user,\
                                      m.version = $version\
                                on match set\
                                      m.country_of_origin = $country_of_origin,\
                                      m.manufacturer_name = $manufacturer_name,\
                                      m.manufacturer_address = $manufacturer_address,\
                                      m.manufacturer_org_type = $manufacturer_org_type,\
                                      m.manufacturer_short_name = $manufacturer_short_name,\
                                      m.source_identifier = $source_identifier,\
                                      m.start_date = $start_date,\
                                      m.end_date = $end_date,\
                                      m.last_update_date = $last_update_date,\
                                      m.last_update_user = $last_update_user,\
                                      m.version = $version",
        manufacturer_id = record['MANUFACTURER_ID'],
        manufacturer_name = record['MANUFACTURER_NAME'],
        country_of_origin = record['COUNTRY_OF_ORIGIN'],
        manufacturer_address = record['MANUFACTURER_ADDRESS'],
        manufacturer_org_type = record['MANUFACTURER_ORG_TYPE'],
        manufacturer_short_name = record['MANUFACTURER_SHORT_NAME'],
        source_identifier = record['SOURCE_IDENTIFIER'],
        start_date = record['START_DATE'],
        end_date = record['END_DATE'],
        last_update_date = record['LAST_UPDATE_DT'],
        last_update_user = record['LAST_UPDATE_USER'],
        version = record['VERSION'])

Create Distributor Node

In [ ]:
for record in df_distributor.rdd.collect():
    graph.run("merge (d: Distributor{distributor_id: $distributor_id})\
                            on create set\
                                     d.country_of_origin = $country_of_origin,\
                                     d.distributor_name = $distributor_name,\
                                     d.distributor_org_type = $distributor_org_type,\
                                     d.distributor_address = $distributor_address,\
                                     d.distributor_short_name = $distributor_short_name,\
                                     d.source_identifier = $source_identifier,\
                                     d.start_date = $start_date,\
                                     d.end_date = $end_date,\
                                     d.last_update_date = $last_update_date,\
                                     d.last_update_user = $last_update_user,\
                                     d.version = $version\
                            on match set\
                                     d.country_of_origin = $country_of_origin,\
                                     d.distributor_name = $distributor_name,\
                                     d.distributor_org_type = $distributor_org_type,\
                                     d.distributor_address = $distributor_address,\
                                     d.distributor_short_name = $distributor_short_name,\
                                     d.source_identifier = $source_identifier,\
                                     d.start_date = $start_date,\
                                     d.end_date = $end_date,\
                                     d.last_update_date = $last_update_date,\
                                     d.last_update_user = $last_update_user,\
                                     d.version = $version",
                distributor_id = record['DISTRIBUTOR_ID'],
                distributor_name = record['DISTRIBUTOR_NAME'],
                country_of_origin = record['COUNTRY_OF_ORIGIN'],
                distributor_org_type = record['DISTRIBUTOR_ORG_TYPE'],
                distributor_address = record['DISTRIBUTOR_ADDRESS'],
                distributor_short_name = record['DISTRIBUTOR_SHORT_NAME'],
                source_identifier = record['SOURCE_IDENTIFIER'],
                start_date = record['START_DATE'],
                end_date = record['END_DATE'],
                last_update_date = record['LAST_UPDATE_DT'],
                last_update_user = record['LAST_UPDATE_USER'],
                version = record['VERSION'])

Create Hospital Node

In [ ]:
for record in df_hospital.rdd.collect():
    graph.run("merge (h: Hospital{hospital_id: $hospital_id})\
                            on create set\
                                  h.hospital_address = $hospital_address,\
                                  h.country_of_origin = $country_of_origin,\
                                  h.hospital_name = $hospital_name, \
                                  h.hospital_short_name = $hospital_short_name,\
                                  h.hospital_type = $hospital_type,\
                                  h.source_identifier = $source_identifier,\
                                  h.start_date = $start_date,\
                                  h.end_date = $end_date,\
                                  h.last_update_date = $last_update_date,\
                                  h.last_update_user = $last_update_user,\
                                  h.version = $version\
                            on match set\
                                  h.hospital_address = $hospital_address,\
                                  h.country_of_origin = $country_of_origin,\
                                  h.hospital_name = $hospital_name, \
                                  h.hospital_short_name = $hospital_short_name,\
                                  h.hospital_type = $hospital_type,\
                                  h.source_identifier = $source_identifier,\
                                  h.start_date = $start_date,\
                                  h.end_date = $end_date,\
                                  h.last_update_date = $last_update_date,\
                                  h.last_update_user = $last_update_user,\
                                  h.version = $version",
            hospital_id = record['HOSPITAL_ID'],
            hospital_name = record['HOSPITAL_NAME'],
            country_of_origin = record['COUNTRY_OF_ORIGIN'],
            hospital_type = record['HOSPITAL_TYPE'],
            hospital_short_name = record['HOSPITAL_SHORT_NAME'],
            hospital_address = record['HOSPITAL_ADDRESS'],
            source_identifier = record['SOURCE_IDENTIFIER'],
            start_date = record['START_DATE'],
            end_date = record['END_DATE'],
            last_update_date = record['LAST_UPDATE_DT'],
            last_update_user = record['LAST_UPDATE_USER'],
            version = record['VERSION'])

Create User Role Node

In [ ]:
for record in df_user_roles.rdd.collect():
    graph.run("merge (u: User_Roles{user_id: $user_id})\
                            on create set\
                                    u.country_of_origin = $country_of_origin,\
                                    u.company_name = $company_name,\
                                    u.designation = $designation,\
                                    u.email_id = $email_id,\
                                    u.first_name = $first_name,\
                                    u.last_name = $last_name,\
                                    u.password = $password,\
                                    u.phone_number = $phone_number,\
                                    u.user_address = $user_address,\
                                    u.user_role = $user_role,\
                                    u.start_date = $start_date,\
                                    u.end_date = $end_date,\
                                    u.last_update_date = $last_update_date,\
                                    u.version = $version\
                            on match set\
                                    u.country_of_origin = $country_of_origin,\
                                    u.company_name = $company_name,\
                                    u.designation = $designation,\
                                    u.email_id = $email_id,\
                                    u.first_name = $first_name,\
                                    u.last_name = $last_name,\
                                    u.password = $password,\
                                    u.phone_number = $phone_number,\
                                    u.user_address = $user_address,\
                                    u.user_role = $user_role,\
                                    u.start_date = $start_date,\
                                    u.end_date = $end_date,\
                                    u.last_update_date = $last_update_date,\
                                    u.version = $version",
             company_name = record['COMPANY_NAME'],
             country_of_origin = record['COUNTRY_OF_ORIGIN'],
             designation = record['DESIGNATION'],
             email_id = record['EMAIL_ID'],
             first_name = record['FIRST_NAME'],
             last_name = record['LAST_NAME'],
             password = record['PASSWORD'],
             phone_number = record['PHONE_NUMBER'],
             user_address = record['USER_ADDRESS'],
             user_id = record['USER_ID'],
             user_role = record['USER_ROLE'],
             start_date = record['START_DATE'],
             end_date = record['END_DATE'],
             last_update_date = record['LAST_UPDATE_DT'],
             version = record['VERSION'])

Create GPO Node

In [ ]:
for record in df_gpo.rdd.collect():
    graph.run("merge (g: GPO{gpo_id: $gpo_id}) \
                            on create set\
                                    g.gpo_name = $gpo_name,\
                                    g.gpo_short_name = $gpo_short_name, \
                                    g.gpo_org_type = $gpo_org_type, \
                                    g.gpo_address = $gpo_address,\
                                    g.country_of_origin = $country_of_origin,\
                                    g.source_identifier = $source_identifier,\
                                    g.start_date = $start_date,\
                                    g.end_date = $end_date,\
                                    g.last_update_date = $last_update_date,\
                                    g.last_update_user = $last_update_user,\
                                    g.version = $version\
                            on match set\
                                    g.gpo_name = $gpo_name,\
                                    g.gpo_short_name = $gpo_short_name, \
                                    g.gpo_org_type = $gpo_org_type, \
                                    g.gpo_address = $gpo_address,\
                                    g.country_of_origin = $country_of_origin,\
                                    g.source_identifier = $source_identifier,\
                                    g.start_date = $start_date,\
                                    g.end_date = $end_date,\
                                    g.last_update_date = $last_update_date,\
                                    g.last_update_user = $last_update_user,\
                                    g.version = $version",
                gpo_id = record['GPO_ID'], 
                gpo_name = record['GPO_NAME'],
                gpo_short_name = record['GPO_SHORT_NAME'],
                country_of_origin = record['COUNTRY_OF_ORIGIN'],
                gpo_org_type = record['GPO_ORG_TYPE'],
                gpo_address = ['GPO_ADDRESS'],
                source_identifier = record['SOURCE_IDENTIFIER'],
                start_date = record['START_DATE'],
                end_date = record['END_DATE'],
                last_update_date = record['LAST_UPDATE_DT'],
                last_update_user = record['LAST_UPDATE_USER'],
                version = record['VERSION'])

Create Correlation Node

In [ ]:
for record in df_correlation_data.rdd.collect():
    graph.run("merge (n:Correlation_Data{product_id: $product_id,\
                                          distributor_name: $distributor_name,\
                                          distributor_sku: $distributor_sku, \
                                          hospital_name: $hospital_name, \
                                          hospital_sku: $hospital_sku,\
                                          manufacturer_name: $manufacturer_name,\
                                          contract_number: $contract_number,\
                                          contract_price: $contract_price,\
                                          base_price: $base_price})",
                distributor_name = record['DISTRIBUTOR_NAME'],
                distributor_sku = record['DISTRIBUTOR_SKU'],
                hospital_name = record['HOSPITAL_NAME'],
                hospital_sku = record['HOSPITAL_SKU'],
                product_id = record['PRODUCT_ID'],
                manufacturer_name = record['MANUFACTURER_NAME'],
                contract_number = record['CONTRACT_NUMBER'],
                contract_price = record['CONTRACT_PRICE'],
                base_price = record['BASE_PRICE'])

Create Product-Similarity Node

In [ ]:
for record in df_product_similarity.rdd.collect():
    graph.run("merge (n: Product_Similarity{manufacturer_name: $manufacturer_name, \
                                            product_id: $product_id, \
                                            product_description: $product_description, \
                                            first_match_id: $first_match_id, \
                                            first_match_description: $first_match_description, \
                                            first_match_score: $first_match_score, \
                                            second_match_id: $second_match_id, \
                                            second_match_description: $second_match_description, \
                                            second_match_score: $second_match_score, \
                                            third_match_id: $third_match_id, \
                                            third_match_description: $third_match_description, \
                                            third_match_score: $third_match_score, \
                                            fourth_match_id: $fourth_match_id, \
                                            fourth_match_description: $fourth_match_description, \
                                            fourth_match_score: $fourth_match_score, \
                                            fifth_match_id: $fifth_match_id, \
                                            fifth_match_description: $fifth_match_description, \
                                            fifth_match_score: $fifth_match_score})",
                manufacturer_name = record['Manufacturer'],
                product_id = record['Product_ID'],
                product_description = record['Product_Description'],
                first_match_id = record['First_Match_ID'],
                first_match_description = record['First_Match_Description'],
                first_match_score = record['First_Match_Score'],
                second_match_id = record['Second_Match_ID'],
                second_match_description = record['Second_Match_Description'],
                second_match_score = record['Second_Match_Score'],
                third_match_id = record['Third_Match_ID'],
                third_match_description = record['Third_Match_Description'],
                third_match_score = record['Third_Match_Score'],
                fourth_match_id = record['Fourth_Match_ID'],
                fourth_match_description = record['Fourth_Match_Description'],
                fourth_match_score = record['Fourth_Match_Score'],
                fifth_match_id = record['Fifth_Match_ID'],
                fifth_match_description = record['Fifth_Match_Description'],
                fifth_match_score = record['Fifth_Match_Score'])

Creating Contract Node

In [ ]:
# GET CONTRACT DATA FROM CSV
df_contract = spark.read.csv('../Data/SampleData/contract.csv', header = True).dropna(how = 'all')   

for record in df_contract.rdd.collect():
    graph.run("merge (n: Contract{contract_number: $contract_number, \
                                  contract_name: $contract_name, \
                                  contract_description: $contract_description, \
                                  start_date: $start_date, \
                                  end_date: $end_date, \
                                  gpo_name: $gpo_name, \
                                  gpo_sign_date: $gpo_sign_date, \
                                  manufacturer_name: $manufacturer_name, \
                                  manufacurer_sign_date: $manufacurer_sign_date, \
                                  distributor_name: $distributor_name, \
                                  distributor_sign_date: $distributor_sign_date, \
                                  hospital_name: $hospital_name, \
                                  hospital_sign_date: $hospital_sign_date})",
                contract_number = record['CONTRACT_NUMBER'], 
                contract_name = record['CONTRACT_NAME'], 
                contract_description = record['CONTRACT_DESCRIPTION'], 
                start_date = record['START_DATE'], 
                end_date = record['END_DATE'], 
                gpo_name = record['GPO_NAME'], 
                gpo_sign_date = record['GPO_SIGN_DATE'], 
                manufacturer_name = record['MANUFACTURER_NAME'], 
                manufacurer_sign_date = record['MANUFACURER_SIGN_DATE'], 
                distributor_name = record['DISTRIBUTOR_NAME'], 
                distributor_sign_date = record['DISTRIBUTOR_SIGN_DATE'], 
                hospital_name = record['HOSPITAL_NAME'], 
                hospital_sign_date = record['HOSPITAL_SIGN_DATE'])

Creating relationships in Neo4J

In [ ]:
graph.run("match (m: Manufacturer), (p: Product) \
           where p.manufacturer = m.manufacturer_name \
           merge (p) - [r: created_by] -> (m)")

graph.run("match (m: Manufacturer), (d: Distributor), (c: Correlation_Data) \
           where c.manufacturer_name = m.manufacturer_name and c.distributor_name = d.distributor_name \
           merge (m) - [r: supplied_to] -> (d)")

graph.run("match (h: Hospital), (d: Distributor), (c: Correlation_Data) \
           where h.hospital_name = c.hospital_name and c.distributor_name = d.distributor_name \
           merge (d) - [r: shipped_to] -> (h)")

graph.run("match (h: Hospital), (p: Product), (c: Correlation_Data) \
           where p.product_id = c.product_id and h.hospital_name = c.hospital_name \
           merge (p) - [r: product_shipped_to{hospital_sku: c.hospital_sku}] -> (h)")

graph.run("match (d: Distributor), (p: Product), (c: Correlation_Data) \
           where p.product_id = c.product_id and c.distributor_name = d.distributor_name \
           merge (p) - [r: product_supplied_to{distributor_sku: c.distributor_sku}] -> (d)")

print('COMPLETED')

In [ ]:
graph.run("match (cn: Contract), (p: Product), (c: Correlation_Data) \
           where p.product_id = c.product_id and c.contract_number = cn.contract_number \
           merge (p) - [r: in_contract] -> (cn)")   

In [ ]:
# ## MATCH p=()-[r:similar_product]->() DETACH DELETE r
### MATCH p=({product_id: "2014"})-[r:similar_product]->() RETURN p LIMIT 25

graph.run("match (p: Product), (ps: Product_Similarity), (p1: Product) \
           where p.product_id = ps.product_id and \
                 p1.product_id = ps.first_match_id and \
                 p.product_id <> p1.product_id and \
                 p.product_id <> ps.first_match_id \
           merge (p) - [r: similar_product{match_score: ps.first_match_score}] -> (p1)")

graph.run("match (p: Product), (ps: Product_Similarity), (p1: Product) \
           where p.product_id = ps.product_id and \
                 p1.product_id = ps.second_match_id and \
                 p.product_id <> p1.product_id and \
                 p.product_id <> ps.second_match_id \
           merge (p) - [r: similar_product{match_score: ps.second_match_score}] -> (p1)")         

graph.run("match (p: Product), (ps: Product_Similarity), (p1: Product) \
           where p.product_id = ps.product_id and \
                 p1.product_id = ps.third_match_id and \
                 p.product_id <> p1.product_id and \
                 p.product_id <> ps.third_match_id \
           merge (p) - [r: similar_product{match_score: ps.third_match_score}] -> (p1)")       

graph.run("match (p: Product), (ps: Product_Similarity), (p1: Product) \
           where p.product_id = ps.product_id and \
                 p1.product_id = ps.fourth_match_id and \
                 p.product_id <> p1.product_id and \
                 p.product_id <> ps.fourth_match_id \
           merge (p) - [r: similar_product{match_score: ps.fourth_match_score}] -> (p1)")               


graph.run("match (p: Product), (ps: Product_Similarity), (p1: Product) \
           where p.product_id = ps.product_id and \
                 p1.product_id = ps.fifth_match_id and \
                 p.product_id <> p1.product_id and \
                 p.product_id <> ps.fifth_match_id \
           merge (p) - [r: similar_product{match_score: ps.fifth_match_score}] -> (p1)")            